In [1]:
# remove hash to install package
# !pip3 install owslib==0.25.0 fiona==1.8.21 geopandas==0.10.2 requests==2.28.0 folium==0.12.1

In [1]:
from owslib.wfs import WebFeatureService
import geopandas
import folium
import io
import zipfile
import pandas as pd
import os
from urllib.request import urlretrieve

## Download external data from AURIN

In [2]:
# User credential to connect with API
WFS_USERNAME = 'ilcag'
WFS_PASSWORD= 'NCpx3cztUjshvRDb'
WFS_URL='https://adp.aurin.org.au/geoserver/wfs'

In [4]:
# Connect with API
adp_client = WebFeatureService(url=WFS_URL,username=WFS_USERNAME, password=WFS_PASSWORD, version='2.0.0')

In [5]:
def download_aurin_df(type_name, file_name):
    """
        This function downloads the dataset from AURIN API
        
        type_name: dataset identifier from the website
        file_name: output file name 
    """

    output_dir = '../data/abs'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    response = adp_client.getfeature(typename=type_name)
    out = open(f'{output_dir}/{file_name}.gml', 'wb')
    out.write(response.read())
    out.close()
    return geopandas.read_file(f'{output_dir}/{file_name}.gml')



In [8]:
# Download selected external data

pop_df = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_3:abs_regional_population_sa2_2001_2021', 
                           '2021_population_census')

In [11]:
sa2_bound = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_GeoLevel:sa2_2016_aust',
                        'sa2_boundaries')

In [14]:
poa_bound = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_GeoLevel:poa_2016_aust',
                        'poa_boundaries')

In [15]:
sa2_income = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_3:abs_personal_income_total_income_distribution_sa2_2017_18', 
                        'sa2_income')

In [16]:
# Select 2021 population census

area_id = ['gml_id', 'primaryindex', 'state_code_2016', 'sa2_maincode_2016',
           'sa2_name_2016']
col_2021 = [x for x in pop_df.columns if '2021' in x or '2020_21' in x]
pop_21 = pop_df[area_id + col_2021]

In [17]:
pop_21.shape

(2292, 18)

There should be 2,310 SA2 regions.

In [18]:
# ERP refers to estimated resident population
pop_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2292 entries, 0 to 2291
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   gml_id                           2292 non-null   object 
 1   primaryindex                     2292 non-null   int64  
 2   state_code_2016                  2292 non-null   int64  
 3   sa2_maincode_2016                2292 non-null   int64  
 4   sa2_name_2016                    2292 non-null   object 
 5   erp_2021                         2292 non-null   int64  
 6   erp_change_number_2020_21        2292 non-null   int64  
 7   erp_change_per_cent_2020_21      2292 non-null   float64
 8   pop_density_2021_people_per_km2  2292 non-null   float64
 9   births_2020_21                   2288 non-null   float64
 10  deaths_2020_21                   2288 non-null   float64
 11  natural_increase_2020_21         2288 non-null   float64
 12  internal_arrivals_20

Do we want to include these regions in the data???
If so, will we include the attributes with missing data?
If yes, how?
If no, why?

In [19]:
pop_21[pop_21["births_2020_21"].isnull()]

,gml_id,primaryindex,state_code_2016,sa2_maincode_2016,sa2_name_2016,erp_2021,erp_change_number_2020_21,erp_change_per_cent_2020_21,pop_density_2021_people_per_km2,births_2020_21,deaths_2020_21,natural_increase_2020_21,internal_arrivals_2020_21,internal_departures_2020_21,net_internal_migration_2020_21,overseas_arrivals_2020_21,overseas_departures_2020_21,net_overseas_migration_2020_21
2231,abs_regional_population_sa2_2001_2021.2292,2292,9,901041004,Norfolk Island,1749,14,0.8069,45.251202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2289,abs_regional_population_sa2_2001_2021.2289,2289,9,901011001,Christmas Island,1979,15,0.7637,14.537000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2290,abs_regional_population_sa2_2001_2021.2290,2290,9,901021002,Cocos (Keeling) Islands,579,6,1.0471,42.212601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2291,abs_regional_population_sa2_2001_2021.2291,2291,9,901031003,Jervis Bay,397,2,0.5063,5.854300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## Download Postcode to SA2 table

In [20]:
def download_url(url, filename):
    '''
        This function downloads data from the specified url.

        url: url of specified website
        filename: output file name
    '''
    output_dir = '../data/abs'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    print(f"Begin downloading {filename} data")
    output_dir = f"{output_dir}/{filename}.zip"
    urlretrieve(url, output_dir)
    print(f"Completed")

In [21]:
download_url('https://www.abs.gov.au/AUSSTATS/subscriber.nsf/log?openagent&1270055006_CG_POSTCODE_2011_SA2_2011.zip&1270.0.55.006&Data%20Cubes&70A3CE8A2E6F9A6BCA257A29001979B2&0&July%202011&27.06.2012&Latest'
             , 'poa_sa2_lookup')

Begin downloading poa_sa2_lookup data
Completed


In [22]:
# Open zipfile
unzip_poa_sa2 = zipfile.ZipFile('../data/abs/poa_sa2_lookup.zip') 

In [23]:
poa_to_sa2 = pd.read_excel(unzip_poa_sa2.open('1270055006_CG_POSTCODE_2011_SA2_2011.xls')
                        , sheet_name='Table 3', skiprows=5)

In [24]:
poa_to_sa2 = poa_to_sa2.dropna()

There are 2162 SA2 codes in Postcode to SA2 data, but there are 2292. Try to find newer data for poa_to_sa2?

In [25]:
poa_to_sa2["SA2_MAINCODE_2011"].unique().size

2162

In [26]:
poa_bound

,gml_id,primaryindex,objectid,poa_code_2016,poa_name_2016,area_albers_sqkm,geometry
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83450 -12.45800, 130.83390 -12.4..."
1,poa_2016_aust.2,2,2,810,0810,23.7902,"POLYGON ((130.84710 -12.37750, 130.84730 -12.3..."
2,poa_2016_aust.3,3,3,812,0812,35.8899,"POLYGON ((130.89190 -12.36880, 130.89220 -12.3..."
3,poa_2016_aust.4,4,4,815,0815,0.6381,"POLYGON ((130.87240 -12.37650, 130.87230 -12.3..."
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83500 -12.43010, 130.83510 -12.4..."
...,...,...,...,...,...,...,...
2665,poa_2016_aust.2639,2639,2639,7268,7268,162.4060,"POLYGON ((147.20120 -41.25830, 147.20130 -41.2..."
2666,poa_2016_aust.2640,2640,2640,7270,7270,265.1620,"MULTIPOLYGON (((146.82300 -41.13960, 146.82290..."
2667,poa_2016_aust.2641,2641,2641,7275,7275,340.3100,"POLYGON ((146.84830 -41.24130, 146.84850 -41.2..."
2668,poa_2016_aust.2642,2642,2642,7276,7276,5.9926,"POLYGON ((146.97820 -41.28340, 146.97820 -41.2..."


In [27]:
sa2_bound

,gml_id,primaryindex,sa2_maincode_2016,sa2_5digitcode_2016,sa2_name_2016,sa3_code_2016,sa3_name_2016,sa4_code_2016,sa4_name_2016,gccsa_code_2016,gccsa_name_2016,state_code_2016,state_name_2016,area_albers_sqkm,geometry
0,sa2_2016_aust.1,1,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,"POLYGON ((149.58420 -35.44430, 149.58440 -35.4..."
1,sa2_2016_aust.2,2,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,"POLYGON ((149.21900 -35.36740, 149.21800 -35.3..."
2,sa2_2016_aust.9,9,101031015,11015,Cooma Region,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6250.8748,"POLYGON ((148.60440 -36.13520, 148.60450 -36.1..."
3,sa2_2016_aust.10,10,101031016,11016,Jindabyne - Berridale,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3939.5484,"POLYGON ((148.27030 -36.46410, 148.27060 -36.4..."
4,sa2_2016_aust.11,11,101041017,11017,Batemans Bay,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,63.7074,"POLYGON ((150.23540 -35.70390, 150.23530 -35.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,sa2_2016_aust.2305,2305,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,"POLYGON ((105.67390 -10.41570, 105.67400 -10.4..."
2306,sa2_2016_aust.2306,2306,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,"MULTIPOLYGON (((96.83050 -12.17640, 96.83050 -..."
2307,sa2_2016_aust.2307,2307,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,"MULTIPOLYGON (((150.69570 -35.18300, 150.69560..."
2308,sa2_2016_aust.2308,2308,901041004,91004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,38.6509,"MULTIPOLYGON (((167.99470 -29.04530, 167.99430..."


In [28]:
poa_bound["geometry"] = poa_bound["geometry"].to_crs(epsg=4326)
sa2_bound["geometry"] = sa2_bound["geometry"].to_crs(epsg=4326)

In [29]:
poa_bound["centroid"] = poa_bound["geometry"].centroid

/var/folders/kn/0h56wsl91tz9kbw4pcdll7zw0000gn/T/ipykernel_2010/1735245261.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poa_bound["centroid"] = poa_bound["geometry"].centroid


In [30]:
poa_bound = poa_bound[poa_bound["centroid"].notnull()]

In [31]:
sa2_bound = sa2_bound[sa2_bound["geometry"].notnull()]

In [32]:
from shapely.validation import make_valid
sa2_bound["geometry"] = sa2_bound["geometry"].apply(make_valid)

/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [45]:
from shapely.geometry import Point
poa_list = poa_bound["centroid"].to_list()
sa2_list = []
for point in poa_list:
    contains = sa2_bound["geometry"].contains(Point(point))
    sa2_list.append(contains.index[contains])

In [34]:
sa2_list

[Int64Index([2130], dtype='int64'),
 Int64Index([1401], dtype='int64'),
 Int64Index([1454], dtype='int64'),
 Int64Index([1401], dtype='int64'),
 Int64Index([2139], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([2117], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([103], dtype='int64'),
 Int64Index([185], dtype='int64'),
 Int64Index([2308], dtype='int64'),
 Int64Index([2259], dtype='int64'),
 Int64Index([2269], dtype='int64'),
 Int64Index([1375], dtype='int64'),
 Int64Index([1943], dtype='int64'),
 Int64Index([1947], dtype='int64'),
 Int64Index([2165], dtype='int64'),
 Int64Index([2120], dtype='int64'),
 Int64Index([2117], dtype='int64'),
 Int64Index([1943], dtype='int64'),
 Int64Index([1946], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([2120], dtype='int64'),
 Int64Index([2120], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([2185], dtype='int64'),
 Int64Index([2176], dtype='int64'),
 Int64Index([2175], dtype='int

In [35]:
poa_bound["sa2_area"] = pd.Series(sa2_list)

In [74]:
sa2_bound

,gml_id,primaryindex,sa2_maincode_2016,sa2_5digitcode_2016,sa2_name_2016,sa3_code_2016,sa3_name_2016,sa4_code_2016,sa4_name_2016,gccsa_code_2016,gccsa_name_2016,state_code_2016,state_name_2016,area_albers_sqkm,geometry
0,sa2_2016_aust.1,1,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,"POLYGON ((149.58421 -35.44429, 149.58441 -35.4..."
1,sa2_2016_aust.2,2,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,"POLYGON ((149.21901 -35.36739, 149.21801 -35.3..."
2,sa2_2016_aust.9,9,101031015,11015,Cooma Region,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6250.8748,GEOMETRYCOLLECTION (POLYGON ((148.60451 -36.13...
3,sa2_2016_aust.10,10,101031016,11016,Jindabyne - Berridale,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3939.5484,"POLYGON ((148.27031 -36.46409, 148.27061 -36.4..."
4,sa2_2016_aust.11,11,101041017,11017,Batemans Bay,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,63.7074,GEOMETRYCOLLECTION (MULTIPOLYGON (((150.14321 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,sa2_2016_aust.2302,2302,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,"POLYGON ((148.80411 -35.37619, 148.80421 -35.3..."
2305,sa2_2016_aust.2305,2305,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,"POLYGON ((105.67391 -10.41569, 105.67401 -10.4..."
2306,sa2_2016_aust.2306,2306,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,GEOMETRYCOLLECTION (MULTIPOLYGON (((96.84181 -...
2307,sa2_2016_aust.2307,2307,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,GEOMETRYCOLLECTION (MULTIPOLYGON (((150.69521 ...


In [76]:
poa_w_sa2 = poa_bound.sjoin(sa2_bound[['sa2_maincode_2016', 'sa2_name_2016','geometry']], lsuffix="sa2_area", rsuffix=sa2_bound.index)

In [78]:
poa_w_sa2.head(20)

,gml_id,primaryindex,objectid,poa_code_2016,poa_name_2016,area_albers_sqkm,geometry,centroid,sa2_area,"index_Int64Index([ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,\n ...\n 2297, 2298, 2299, 2300, 2301, 2302, 2305, 2306, 2307, 2308],\n dtype='int64', length=2292)",sa2_maincode_2016,sa2_name_2016
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83451 -12.45799, 130.83391 -12.4...",POINT (130.84373 -12.46185),"Int64Index([2130], dtype='int64')",2130,701011002,Darwin City
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83501 -12.43009, 130.83511 -12.4...",POINT (130.86497 -12.42338),"Int64Index([2139], dtype='int64')",2130,701011002,Darwin City
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83451 -12.45799, 130.83391 -12.4...",POINT (130.84373 -12.46185),"Int64Index([2130], dtype='int64')",2138,701011005,Larrakeyah
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83501 -12.43009, 130.83511 -12.4...",POINT (130.86497 -12.42338),"Int64Index([2139], dtype='int64')",2138,701011005,Larrakeyah
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83451 -12.45799, 130.83391 -12.4...",POINT (130.84373 -12.46185),"Int64Index([2130], dtype='int64')",2141,701011008,Stuart Park
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83501 -12.43009, 130.83511 -12.4...",POINT (130.86497 -12.42338),"Int64Index([2139], dtype='int64')",2141,701011008,Stuart Park
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83451 -12.45799, 130.83391 -12.4...",POINT (130.84373 -12.46185),"Int64Index([2130], dtype='int64')",2137,701011004,Fannie Bay - The Gardens
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83501 -12.43009, 130.83511 -12.4...",POINT (130.86497 -12.42338),"Int64Index([2139], dtype='int64')",2137,701011004,Fannie Bay - The Gardens
1,poa_2016_aust.2,2,2,810,0810,23.7902,"POLYGON ((130.84711 -12.37749, 130.84731 -12.3...",POINT (130.87685 -12.36972),"Int64Index([1401], dtype='int64')",2139,701011006,Ludmilla - The Narrows
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83501 -12.43009, 130.83511 -12.4...",POINT (130.86497 -12.42338),"Int64Index([2139], dtype='int64')",2139,701011006,Ludmilla - The Narrows


In [37]:
poa_bound

,gml_id,primaryindex,objectid,poa_code_2016,poa_name_2016,area_albers_sqkm,geometry,centroid,sa2_area
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83451 -12.45799, 130.83391 -12.4...",POINT (130.84373 -12.46185),"Int64Index([2130], dtype='int64')"
1,poa_2016_aust.2,2,2,810,0810,23.7902,"POLYGON ((130.84711 -12.37749, 130.84731 -12.3...",POINT (130.87685 -12.36972),"Int64Index([1401], dtype='int64')"
2,poa_2016_aust.3,3,3,812,0812,35.8899,"POLYGON ((130.89191 -12.36879, 130.89221 -12.3...",POINT (130.91450 -12.37765),"Int64Index([1454], dtype='int64')"
3,poa_2016_aust.4,4,4,815,0815,0.6381,"POLYGON ((130.87241 -12.37649, 130.87231 -12.3...",POINT (130.86899 -12.37174),"Int64Index([1401], dtype='int64')"
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83501 -12.43009, 130.83511 -12.4...",POINT (130.86497 -12.42338),"Int64Index([2139], dtype='int64')"
...,...,...,...,...,...,...,...,...,...
2665,poa_2016_aust.2639,2639,2639,7268,7268,162.4060,"POLYGON ((147.20121 -41.25829, 147.20131 -41.2...",POINT (147.21952 -41.29213),"Int64Index([2135], dtype='int64')"
2666,poa_2016_aust.2640,2640,2640,7270,7270,265.1620,"MULTIPOLYGON (((146.82301 -41.13959, 146.82291...",POINT (146.78669 -41.17828),"Int64Index([2135], dtype='int64')"
2667,poa_2016_aust.2641,2641,2641,7275,7275,340.3100,"POLYGON ((146.84831 -41.24129, 146.84851 -41.2...",POINT (146.81144 -41.31036),"Int64Index([2135], dtype='int64')"
2668,poa_2016_aust.2642,2642,2642,7276,7276,5.9926,"POLYGON ((146.97821 -41.28339, 146.97821 -41.2...",POINT (146.96355 -41.28260),NaN


In [46]:
# check the postcodes not allocated
undef_poa_ind = []
for i in range(len(sa2_list)):
    try:
        sa2_list[i] = sa2_list[i][0]
    except IndexError:
        undef_poa_ind.append(i)
        print(i)

38
66
138
206
335
347
349
352
1408
1539
1784
1863
2160
2302
2492
2496
2524
2528
2531
2567
2603
2626


In [52]:
df_undef_poa = (poa_bound[poa_bound.index.isin(undef_poa_ind)]
                [['poa_code_2016', 'poa_name_2016', 'sa2_area']])

In [53]:
df_undef_poa.head()

,poa_code_2016,poa_name_2016,sa2_area
38,880,0880,"Int64Index([], dtype='int64')"
66,2027,2027,"Int64Index([], dtype='int64')"
138,2105,2105,"Int64Index([], dtype='int64')"
206,5575,5575,"Int64Index([], dtype='int64')"
335,2230,2230,"Int64Index([], dtype='int64')"


In [70]:
# there are postcodes with multiple SA2s
display(poa_to_sa2.sort_values(by=['POSTCODE', 'PERCENTAGE'], ascending=True).head(20))

,POSTCODE,POSTCODE.1,SA2_MAINCODE_2011,SA2_NAME_2011,RATIO,PERCENTAGE
1,0800,800.0,701011002.0,Darwin City,1.000000,99.999998
13,0810,810.0,701021029.0,Wanguri,0.060207,6.020652
5,0810,810.0,701021018.0,Jingili,0.061562,6.156198
8,0810,810.0,701021024.0,Moil,0.068575,6.857527
6,0810,810.0,701021021.0,Lyons (NT),0.070201,7.020120
12,0810,810.0,701021028.0,Wagaman,0.071421,7.142110
2,0810,810.0,701021010.0,Alawa,0.071997,7.199707
7,0810,810.0,701021023.0,Millner,0.083483,8.348290
11,0810,810.0,701021027.0,Tiwi,0.085041,8.504137
3,0810,810.0,701021013.0,Brinkin - Nakara,0.096392,9.639178


In [72]:
# only get the SA2 with highest percentage
idx = poa_to_sa2.groupby(['POSTCODE'])['PERCENTAGE'].transform(max) == poa_to_sa2['PERCENTAGE']
high_perc_sa2_poa = poa_to_sa2[idx].reset_index(drop=True)

In [73]:
pd.merge(df_undef_poa, high_perc_sa2_poa[['POSTCODE', 'SA2_MAINCODE_2011', 'SA2_NAME_2011']], how='left', left_on='poa_name_2016', right_on='POSTCODE')

,poa_code_2016,poa_name_2016,sa2_area,POSTCODE,SA2_MAINCODE_2011,SA2_NAME_2011
0,880,0880,"Int64Index([], dtype='int64')",0880,702041064.0,Nhulunbuy
1,2027,2027,"Int64Index([], dtype='int64')",2027,118011343.0,Double Bay - Bellevue Hill
2,2105,2105,"Int64Index([], dtype='int64')",2105,122021421.0,Bayview - Elanora Heights
3,5575,5575,"Int64Index([], dtype='int64')",5575,405041127.0,Yorke Peninsula - North
4,2230,2230,"Int64Index([], dtype='int64')",2230,128011528.0,Cronulla - Kurnell - Bundeena
5,2261,2261,"Int64Index([], dtype='int64')",2261,102021044.0,Bateau Bay - Killarney Vale
6,2263,2263,"Int64Index([], dtype='int64')",2263,102021048.0,Gorokan - Kanwal - Charmhaven
7,2267,2267,"Int64Index([], dtype='int64')",2267,111021220.0,Wangi Wangi - Rathmines
8,3960,3960,"Int64Index([], dtype='int64')",3960,205031087.0,Foster
9,4178,4178,"Int64Index([], dtype='int64')",4178,301031020.0,Wynnum


All not allocated POAs were found using the 2011 data.